In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import warnings 
from io import StringIO
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup
import nltk #to use the stopwords library
import string # for a list of all punctuation
from nltk.corpus import stopwords # for a list of stopwords

# Preprocessing- Text Mining

#### In this part we will analyse the pdf file and the grievance description to extract the words for our doc2vec model 
### Every neural network architecture requires labels, otherwise it wouldn’t be able to correct its weights. An unsupervised learning algorithm simply finds a way to construct those labels by itself. 

#### Word2Vec, Doc2Vec and Glove are semi-supervised learning algorithms and they are Neural Word Embeddings for the sole purpose of Natural Language Processing.

#### Since this are no labels here for us to determine the classification we then will use this Doc2Vecmodel for our Kmeans algorithm to classify between 1, 0 as the two different parts of the description column

#####  We wil be using the Tika module from Python to read into the pdf file

In [ ]:
##change the source file path when you run the program on another system 
from tika import parser
parsed = parser.from_file('C:\\Users\\IBM_ADMIN\\Desktop\\Phd_sat\\source\\ftca.pdf')
print(parsed["content"])

In [ ]:
new_text = parsed['content']

In [ ]:
# Keeping only the Alphabets and removing .,! etc using regular expression
text_only = re.sub("[^a-zA-Z]", " ", new_text)
print(text_only)

In [ ]:
type(text_only)

In [ ]:
texts=StringIO(text_only).getvalue()

In [ ]:
texts=texts.lower()
texts=texts.split()

### Additional exploratory to find the most frequent words in pdf , to learn what makes this pdf so special that we can understand the fair and unfair from this and make this useful for our Grievance data

#### Experiment ZOne -1
(delete this before sending)

In [ ]:
# Next we only want valid strings
import collections as co
from IPython.display import display, HTML, Markdown, display
#define fucntion to assign Markdown 
def printmd(string):
    display(Markdown(string))
alphaLev = .5

words = co.Counter(nltk.corpus.words.words())
stopWords =co.Counter( nltk.corpus.stopwords.words() )
texts=[i for i in texts if i in words and i not in stopWords]
c = co.Counter(texts)
printmd("We see that we $" + str(len(texts)) + "$ legal word tokens in our corpus. There are $" + str(
        len(list(c.most_common())))
       + "$ legal non-stopword types in our corpus.")

In [ ]:
type(c)

In [ ]:
d = {}
for key, value in c.items():
    d[key] = value

In [ ]:
print(d)

In [ ]:
freq_words = list(d.keys())
print (freq_words)
print(len(freq_words))

In [ ]:
wordFrequencyFrame = pd.DataFrame(c.most_common(len(c)),columns = ["Word","Frequency"])
#plot frequency on rank
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.set_size_inches(18, 7)
#freq-rank
ax1.plot(wordFrequencyFrame.index,wordFrequencyFrame["Frequency"])
ax1.set_title("Frequency on Rank of Vocabulary")
ax1.set_xlabel("Rank")
ax1.set_ylabel("Frequency")
#freq-logRank
ax2.plot(np.log(wordFrequencyFrame.index + 1),np.log(wordFrequencyFrame["Frequency"]))
ax2.set_title("Log-Frequency on Log-Rank of Vocabulary")
ax2.set_xlabel("Log Rank")
ax2.set_ylabel("Frequency")
plt.show()
printmd("_Figure 1: Frequency-Rank Graphs of Our Vocabulary._")
#get 20 most common
top20FrequencyFrame = wordFrequencyFrame.iloc[0:20,:]
display(top20FrequencyFrame)
printmd("_Table 1: The $20$ most frequent words with their frequencies_")
#get 20 least common
bottom20FrequencyFrame = wordFrequencyFrame.iloc[(wordFrequencyFrame.shape[0]-20):wordFrequencyFrame.shape[0],:]
display(bottom20FrequencyFrame)
printmd("_Table 2: The $20$ least frequent words with their frequencies_")

In [ ]:
list_alpha = c.most_common()
print (len(list_alpha))
type(list_alpha)


In [ ]:
from wordcloud import WordCloud

# Read the whole text.
text = str(list_alpha)

# Generate a word cloud image
wordcloud = WordCloud().generate(text)

# Display the generated image:
# the matplotlib way:
import matplotlib.pyplot as plt


# take relative word frequencies into account, lower max_font_size
wordcloud = WordCloud(background_color="white",max_words=len(texts),max_font_size=40, relative_scaling=.9).generate(text)
plt.figure()
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

### the above cloud shows some shady words , thus it seems useful

In [ ]:
##Experiment ZOne -Ends

#### now let us get the grievance description sorted

In [ ]:
data = pd.read_csv("GrievancesData_Train.csv", sep=",",header='infer')

In [ ]:
data_txt = data['GrievanceDescription']

In [ ]:
# Stemming is a process which treats strings like message, messages, messaging as a single word
# Using SnowballStemmer to do the process here
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [ ]:
stemmer = SnowballStemmer("english")

stopwords_ = set(stopwords.words("english"))
# All the process is combined in this function convert_reviews_to_words
def convert_comments_to_words(review):
    review_text = BeautifulSoup(review, "html.parser").get_text()
    alphabets_only = re.sub("[^a-zA-Z]", " ", review_text)
    alphabets_only = re.sub(r'\S*(x{2,}|X{2,})\S*'," ",alphabets_only)
    alphabets_only = re.sub(r'[^\x00-\x7F]+', " ",alphabets_only)
    words = alphabets_only.lower().split()
    meaningful_words = [w for w in words if not w in stopwords_]
    stemmed_words = [stemmer.stem(w) for w in meaningful_words]
    print(stemmed_words)
    return(" ".join(stemmed_words))

In [ ]:
num_grievances = data['GrievanceDescription'].size
print(num_grievances)

In [ ]:
# Initialize an empty list to hold the clean reviews

data_text= []

# Calling the convert_comments_to_words function for every comment in the dataset
# Appending those values to data list
print("Cleaning and parsing the comments from data....")
for i in range(0, num_grievances):
    
        
    data_text.append(convert_comments_to_words(data["GrievanceDescription"][i]))
    
    
print("Cleaning and Parsing complete.")

In [ ]:
print(data_text[:][1])
type(data_text)

## Time to build the semi-supervised model

### Model

Building the Vocabulary Table
Doc2Vec requires us to build the vocabulary table (simply digesting all the words and filtering out the unique words, and doing some basic counts on them). So we feed it the array of sentences. model.build_vocab takes an array of TaggedLineSentence, hence our to_array function in the TaggedLineSentence class. If you're curious about the parameters, do read the Word2Vec documentation. Otherwise, here's a quick rundown:

###### min_count: 
ignore all words with total frequency lower than this. You have to set this to 1, since the sentence labels only appear once. Setting it any higher than 1 will miss out on the sentences.

###### window: 
the maximum distance between the current and predicted word within a sentence. Word2Vec uses a skip-gram model, and this is simply the window size of the skip-gram model.

##### size: 

dimensionality of the feature vectors in output. 100 is a good number. If you're extreme, you can go up to around 400.
###### sample: 
threshold for configuring which higher-frequency words are randomly downsampled
###### workers: 
use this many worker threads to train the model

In [ ]:
import gensim
from gensim.test.utils import common_corpus
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim import models

In [ ]:
type(texts)
type(data_text)

In [ ]:
documents1 = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]

In [ ]:
documents2 = [TaggedDocument(doc, [i]) for i, doc in enumerate(data_text)]

In [ ]:
modeltr = gensim.models.doc2vec.Doc2Vec(documents1 , size = 100,window =2,min_count=1,workers=4)

In [ ]:
docxx = documents1+documents2

In [ ]:
modeltr2 = gensim.models.doc2vec.Doc2Vec(docxx, size = 100,window =2,min_count=1,workers=4)

In [ ]:
#save the doc2vec model
from gensim.test.utils import get_tmpfile
modname = get_tmpfile("my_model")

In [ ]:
#load model from saved file
modeltr2.save(modname)
modeltr2 = Doc2Vec.load(modname) 

Using Pretrained doc2vec Model for Text Clustering (Birch Algorithm)
In this example we use Birch clustering algorithm for clustering 
Birch is unsupervised algorithm that is used for hierarchical clustering. An advantage of this algorithm is its ability to incrementally and dynamically cluster incoming data

We use the following steps here:

Load doc2vec model
Load text docs that will be clustered
Convert docs to vectors (infer_vector)
Do clustering

In [ ]:
from sklearn import metrics
 
import gensim.models as g
import codecs

In [ ]:
model = modeltr2
test_docs = data_text

In [ ]:
#inference hyper-parameters
start_alpha=0.01
infer_epoch=1000

In [ ]:
X=[]
for d in test_docs:
     
    X.append( modeltr2.infer_vector(d, alpha=start_alpha, steps=infer_epoch) )

In [ ]:
X

In [ ]:
X_chunk = X[25000]

In [ ]:
from sklearn.cluster import KMeans

kmeans_model = KMeans(n_clusters=2, init='k-means++', max_iter=100)
kmeans_model.fit(X)



In [ ]:
labels=kmeans_model.labels_.tolist()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2).fit(X)
datapoint = pca.transform(X)

In [ ]:
plt.figure
label1 = ["#FFFF00", "#008000"]
color = [label1[i] for i in labels]
plt.scatter(datapoint[:, 0], datapoint[:, 1], c=color)

centroids = kmeans_model.cluster_centers_
centroidpoint = pca.transform(centroids)
plt.scatter(centroidpoint[:, 0], centroidpoint[:, 1], marker='^', s=150, c='#000000')
plt.show()

In [ ]:
print(len(labels))

In [ ]:
np.unique(labels)

In [ ]:
data['Labels'] = labels

In [ ]:
data.loc[ : , 'Labels'] = labels

In [ ]:
data

In [ ]:
data['Labels'].value_counts()

### 1- Fair == UFDP
### 0- NFair == NUFDP